# Lab 4

Introduction to  EEG Analysis. This notebook is for you to get exposure to EEG signal analysis and artifact removal.

## EEG Theory

### Eletroencephalography

The electroencephalogram or EEG is a recording of the biopotentials in the cerebrum of thebrain. These potentials are typical recorded at the surface of the scalp and can vary withrespect to the emotional, mental, and physiological state of a person. The action potentialsand synaptic potentials of an individual neurons are too small to be measured by electrodes.Therefore, an EEG is a measurement of the summation of the electrical signals produce byneurons in a defined area and over a specific amount of time. It is important to note that theseneurons need not synchronized but may be producing signals in an asynchronous manner.EEG signals can be categorizedby the four major frequency ranges or brainwaves in whichthey occur: alpha, beta, delta and theta. The corresponding frequencies, amplitudes, andtypical human functionality of the waves are seen in Table.


| Brainwave | Frequency | Amplitude (uV) | Human Function |
| --------- | --------- | -------------- | -------------- |
| alpha | 8 - 13 | 2 - 100 | Awake, Quiet, Resting, Eyes Open|
| beta | 13 - 22 | 5 - 100 | Mental Activity or External Stimulus|
| delta | 0.4 - 4 | 20 - 100| Sleep |
| theta | 4 - 8 | 10 | Emotional Stress | 


A special system of electrode placement called the 10-20 system is used during EEG recordings.The 10 and 20 refer to percent distances of the electrodes from each other with respect to the sizeof the patientâ€™s head. The letters F, T, C, P, and O in the 10 20 system refer to frontal, temporal,central, parietal, and occipital or essential lobes of the brain excluding central. Also, even numbers are located on the right hemisphere and odd numbers on the left hemisphere. The letterz is an indicator of the central line of the head.



In [ ]:
'''You will need to install the following package'''
!pip3 install fooof

## Import the CSV
We first need to read in the csv file name from your data. Please insert the name of your file in the code below:

In [ ]:
import pandas as pd
import mne
import os
import scipy
# Opens up another window with your plot
%matplotlib qt 
# Read in Data
directory = 'Lab_4_Data'
participant = ...
file = ...
EEGCsv = pd.read_csv(os.path.join(directory, participant, file))
EEGCsv.index = pd.to_datetime(EEGCsv['timestamps'], unit='s')

# Set channel sampling frequency
fs = 250
seconds = EEGCsv.timestamps- EEGCsv.timestamps.iloc[0]

ch_names = [ 'Fz', 'C3', 'Cz', 'C4', 'Pz', 'PO7', 'Oz', 'PO8'] # set EEG channel names
for ch in [ 'Fz', 'C3', 'Cz', 'C4', 'Pz', 'PO7', 'Oz', 'PO8']:
    EEGCsv[ch] = EEGCsv[ch]/100000 # rescale data (uv to volts)
    EEGCsv[ch] = mne.filter.filter_data(EEGCsv[ch].values, 250, 0.1, 100) # apply a highpass filter
    EEGCsv[ch] = mne.baseline.rescale(EEGCsv[ch].values, seconds, (0,10)) # apply baseline rescaling
    


display(EEGCsv)


## EEG Signals
The above code prints out a table containing your collected data. One thing to note is the column names and what they correspond to. We have time, electrode position values, and Stim (0 or 1). For now, lets focus on the electrode positions.

The data was collected using the standard 10-20 placement. The code below shows you these positions.


In [ ]:
mon = mne.channels.make_standard_montage('standard_1020')
mon.plot(kind='topomap', show_names=True)# 2d Plot

In [ ]:
# The window that this opens is interactable, so you can rotate around to get a better idea where each electrode is.
mon.plot(kind='3d', show_names=True)# 3d Plot

We now need to plot signals from our dataset. Unfortunately, some of the electrode positions are incorrectly named and need to be renamed. Our analysis uses the MNE toolbox. Feel free to look at the MNE documentation online.

# Plot the raw data


In [ ]:
# Create MNE Info to apply to csv data
info = mne.create_info(ch_names, fs, ch_types='eeg')

df_mne = EEGCsv[ch_names]

# Create the Raw Object for MNE
raw = mne.io.RawArray(df_mne.values.transpose(), info)


# Plot the data
raw.plot(n_channels=8)

'''You can scale the graphs using +/-'''

## Plot the PSD

In [ ]:
raw.plot_psd(area_mode='range', tmax=10.0, show=False, average=True)

## Notch Filter

You should see a spike in the PSD ~60. The code below will apply a notch filter at 60 Hz.

In [ ]:
import scipy
import scipy.fftpack
import matplotlib.pyplot as plt
import numpy as np
# Run the following to create a notch filter at 60 hz and view the resulting PSD
raw.notch_filter(np.arange(60, 128, 60))
raw.plot_psd(area_mode='range', tmax=10.0, show=False, average=True)

In [ ]:
def check_nans(data, nan_policy='zero'):
    """Check an array for nan values, and replace, based on policy."""

    # Find where there are nan values in the data
    nan_inds = np.where(np.isnan(data))

    # Apply desired nan policy to data
    if nan_policy == 'zero':
        data[nan_inds] = 0
    elif nan_policy == 'mean':
        data[nan_inds] = np.nanmean(data)
    else:
        raise ValueError('Nan policy not understood.')

    return data

In [ ]:
from mne.viz import plot_topomap
from mne.time_frequency import psd_welch

# FOOOF imports
from fooof import FOOOFGroup
from fooof.bands import Bands
from fooof.analysis import get_band_peak_fg
from fooof.plts.spectra import plot_spectrum

from matplotlib import cm, colors, colorbar

'''Set the 10-20 montage for our data'''
raw.set_montage(mon)

'''Apply the PSD Welch algorithm to get the Power Spectral Density'''
spectra, freqs = psd_welch(raw, fmin=1, fmax=40, tmin=0, tmax=250,
                           n_overlap=150, n_fft=250)

# Initialize a FOOOFGroup object, with desired settings
fg = FOOOFGroup(peak_width_limits=[1, 6], min_peak_height=0.15,
                peak_threshold=2., max_n_peaks=6, verbose=False)

# Define the frequency range to fit
freq_range = [1, 30]

# Fit the power spectrum model across all channels
fg.fit(freqs, spectra, freq_range)

# Define frequency bands of interest
bands = Bands({'theta': [3, 7],
               'alpha': [7, 14],
               'beta': [15, 30]})

# Extract alpha peaks
alphas = get_band_peak_fg(fg, bands.alpha)

# Extract the power values from the detected peaks
alpha_pw = alphas[:, 1]

# We need to remove nans for the plot to show
alpha_pw = check_nans(alpha_pw) 

# Plot the topography of alpha power
plot_topomap(alpha_pw, raw.info, cmap=cm.viridis, contours=0);


In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 6))
for ind, (label, band_def) in enumerate(bands):

    # Get the power values across channels for the current band
    band_power = check_nans(get_band_peak_fg(fg, band_def)[:, 1])

    # Extracted and plot the power spectrum model with the most band power
    fg.get_fooof(np.argmax(band_power)).plot(ax=axes[ind], add_legend=False)

    # Set some plot aesthetics & plot title
    axes[ind].yaxis.set_ticklabels([])
    axes[ind].set_title('Largest ' + label + ' peak', {'fontsize' : 16})

## Event Related Potentials

Now that we have an idea of how to extract some meaningful information from EEG signals, we need to do some analysis specifically with our dataset. 

Cats and dogs flashed on the screen. The participant is looking for a specific stimulus, hence the 'stim' column. These stimuli produce what is called a P300 Event Related Potential. We are going to analyze these potentials between the target and non target stimuli.


In [ ]:
# We are going to go back to the original dataset and use the Pandas Groupby method to figure out when the target events occur
EEGCsv.groupby('stim').get_group(1)

In [ ]:
import datetime as dt
# We can now compare an ERP between a target and non-target event.
# We need to slice our dataframe to get the P300 timeframe from these events.
# P300 events occur when a positive deflection occurs approx 300 msec after a triggering event (stimulus)
# Lets see if we can find this peak visually

# We first find a target time by using the .iloc method
TargetTime = EEGCsv.groupby('stim').get_group(1).index[0]

# Then we can index the TimeFrame for the P300
StartTime = TargetTime - dt.timedelta(200*0.001) # Starting at -200 msec
EndTime   = TargetTime + dt.timedelta(500*0.001) # Ending at 500 msec

ERPdf = EEGCsv[(EEGCsv.index > StartTime) & (EEGCsv.index < EndTime)]

# Lets look at what the raw data looks like for 1 ERP
display(ERPdf)

# This code will be useful for the lab questions when computing the PSD of the ERPs. The next code sections are better for plotting

### MNE Library and ERPs
We first need to drop columns from our raw data that are not EEG based. Then we can scale and create a raw MNE array for easy plotting.

The next step creates Event Info from our Event column in the original array. We can then use MNE functionality to plot the ERPs between target and non-target events.

In [ ]:
# Create MNE Info to apply to csv data
info = mne.create_info(ch_names, fs, ch_types='eeg')
info['chs'][-1]['kind']



# Create Raw Array
raw = mne.io.RawArray(df_mne.transpose(), info)


# Create Event info and add to raw array
info = mne.create_info(['STI 0'], raw.info['sfreq'], ['stim'])
temp = EEGCsv['stim'].values
temp = np.reshape(temp, (1, -1))
stim_raw = mne.io.RawArray(temp, info)
raw.add_channels([stim_raw], force_update_info=True)

# Get events object for easy access
events = mne.find_events(raw, stim_channel='STI 0', verbose=True)

display(events)

# Plot the data
raw.plot(events = events, n_channels=8)

# Note the vertical lines indicating events

In [ ]:
# Set epoch length to -0.2 seconds before stimulus and 0.5 after stimulus
tmin, tmax = -0.2, 0.5

# Set the epoch parameters based on the events in the raw data
epochs_params = dict(events=events, event_id=, tmin=tmin, tmax=tmax)

# Now we parse our data based on the target events and take the average of all channels
epochs = mne.Epochs(raw, **epochs_params).average()
# Set montage to 10-20
epochs.set_montage(mon)
# Plot EEG data and a topomap
display(epochs.plot())
epochs.plot_topomap()

## We can do this for our dog pictures as well

In [ ]:
# Set epoch length to -0.2 seconds before stimulus and 0.5 after stimulus
tmin, tmax = -0.2, 0.5


# Set the epoch parameters based on the events in the raw data
epochs_params = dict(events=events, event_id=2, tmin=tmin, tmax=tmax)

# Now we parse our data based on the target events and take the average of all channels
epochs = mne.Epochs(raw, **epochs_params).average()
# Set montage to 10-20
epochs.set_montage(mon)
# Plot EEG data and a topomap
display(epochs.plot())
epochs.plot_topomap()

We now have two plots for target and non-target events. Note the difference between the two graphs.

## Your Turn

The above code allows you to get a feel for some EEG analysis for ERPs. Please use the code above as reference to complete the following:

### All data analysis
The following should be applied to all of your collected data from the Cats and Dog experiment.
1. Apply a notch filter
2. Compute the average Alpha, Beta, and Theta Power
3. Plot the topomaps for each band (Alpha, Beta, and Theta)

### Target and Non Target data analysis
For each participant
1. Use the notch filtered Raw data from above
2. Compute the average Alpha, Beta, and Theta PSDs for the dog and cat events
3. Plot the Raw Averaged Epoched data for the dog and cat events
4. Plot a Topomap for the Raw Averaged Epoched Data for the dog and cat events

 